In [ ]:
#Análisis del dataset del precio de carros según sus características y creación de un modelo predictivo del precio según sus características más relevantes

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
#Primero leemos los datos
df=pd.read_csv('../input/car-price-prediction/CarPrice_Assignment.csv')
#Mostramos el dataset
df.head()

In [ ]:
#Mostramos un resumen estadístico del dataset
df.describe()

Vemos que hay 205 registros de carros con sus características

In [ ]:
#Vemos si existen missing values
df.isnull().sum()

Vemos que no existen missing values

In [ ]:
#Debido a que son muchas variables, haremos un mapa de calor de correlación para ver qué variables están fuertemente correlacionadas a el precio
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 6))
sns.heatmap(df.corr()[['price']].sort_values(by='price', ascending=False),vmin=-1, vmax=1, annot=True)
plt.title('Mapa de correlación')

Podemos apreciar que las variables que influyen en el precio son: Enginesize, curbweight,horsepower,cardwidth y carlenght. Trabajaremos con estas variables para predecir el precio de nuevos carros. Se realizará una regresión lineal múltiple.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#creamos un modelo lineal
lm=LinearRegression()
#Seleccionamos las características
x_features=df[['enginesize','curbweight','horsepower','carlength','carwidth']]
#dividimos la data en entreamiento y prueba usando el método train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_features,df['price'],test_size=0.3, random_state=0)
#Ajustamos el modelo lineal con los datos de entreamiento
lm.fit(x_train,y_train)
#Creamos una variable de precio predicha con los datos de prueba
yhat=lm.predict(x_test)
#Realizamos una evaluación métrica
RSQU=lm.score(x_test,y_test)
print ("El R_cuadrado es: ", RSQU)
MSE= mean_squared_error(y_test,yhat)
print ("El MSE es: ", MSE)

Vemos qeu el R cuadrado es 0.8, el cual es un valor cercano a 1. Podemos decir que el modelo se ajusta bien para predecir el valor del precio. Ahora graficaremos los valores predichos con los valores reales.

In [ ]:
plt.figure()
ax1=sns.distplot(y_test,hist=False, color = 'r', label="Valores Reales")
sns.distplot(yhat, hist=False, color = 'b', label="Valores Predichos",ax=ax1)

plt.title('Comparación entre el modelo y los datos reales')
plt.ylabel('Proporcion de carros')
plt.xlabel('Precio')
plt.legend()
plt.show()
plt.close()

Como podemos ver, el modelo intenta seguir lo mejor que puede a los valores reales. Vamos a compararlo con una ajuste polinomial de grado 2 y comparar las métricas Rcuadrado y MSE

Usaremos pipeline para normalizar, hacer una transformación polinómica y finalmente una regresión lineal

In [ ]:
#Importamos las librerías necesarias
from sklearn.preprocessing import PolynomialFeatures
#from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


input=[('scale',StandardScaler()),('polynomial',PolynomialFeatures(include_bias=False,degree=2)),('model',LinearRegression())]
pipe=Pipeline(input)
pipe.fit(x_train,y_train)
yhat=pipe.predict(x_test)

#Ahora graficamos
plt.figure()

ax1=sns.distplot(y_test, hist=False, color = 'b', label = "Valores Reales de Prueba")
sns.distplot(yhat,hist=False, color = 'r', label="Valores Predichos",ax=ax1)
plt.title('Comparación entre el modelo y los datos reales')
plt.ylabel('Proporcion de carros')
plt.xlabel('Precio')
plt.legend()
plt.show()
plt.close()

In [ ]:
RSQU=pipe.score(x_test,y_test)
print ("El R_cuadrado es: ", RSQU)
MSE=mean_squared_error(y_test,yhat)
print ("El MSE es: ", MSE)

Podemos comparar el ajuste polinomial y la regresión lineal múltiple (MLR). Podemos ver que el MLR tiene mejor R_CUADRADO y menor MSE que el ajuste polinomial por lo que el MLR se ajusta mejor para predecir el precio de los carros nuevos.